In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'

train_df = pd.read_pickle(PATH+'train_merge.pkl')
test_df = pd.read_pickle(PATH+'test_merge.pkl')
print('done')
print(train_df.shape,test_df.shape)

done
(307511, 899) (48744, 898)


In [2]:
old_merge = pd.read_csv(PATH+'submission/train_ratio_1.csv')

In [15]:
half = old_merge[old_merge['TARGET'] > 0.5]
index_half = half.index

In [12]:
# Find all positions for those, and drop the rows for the test, concat for train, and set target to be 1


In [16]:
test_half = test_df.iloc[index_half]
test_half['TARGET'] = 1

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
test_half.shape

(173, 899)

In [20]:
test_df.drop(index_half, inplace = True)

In [21]:
test_df.shape

(48571, 898)

In [22]:
train = pd.concat([train_df,test_half])

In [23]:
train.shape

(307684, 899)

In [26]:
train.to_pickle(PATH + 'train_798.pkl')
test_df.to_pickle(PATH + 'test_798.pkl')

In [1]:
half

NameError: name 'half' is not defined